In [ ]:
import cv2 
import tensorflow as tf 
import numpy as np 
from sklearn import metrics
from sklearn.metrics import confusion_matrix,f1_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import tensorflow.keras
from imblearn.over_sampling import SMOTE
from tensorflow.keras.layers import Conv2D, BatchNormalization,MaxPool2D, GlobalMaxPool2D,LSTM,Bidirectional
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential, load_model
from imblearn.under_sampling import RandomUnderSampler
import tensorflow.keras
from tensorflow.keras.layers import Conv2D, BatchNormalization,MaxPool2D, GlobalMaxPool2D,LSTM,Bidirectional
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout

In [ ]:
import tensorflow as tf

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# Check if GPU is available
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


In [ ]:
#for fakeavceleb dataset
rnn_features=np.load('F:\\fakeavceleb_balance\\data npy format\\bi-lstm\\fakeavceleb_bi_conv_lstm_features.npy')

In [ ]:
#for fakeavceleb dataset
spec_features=np.load('F:\\fakeavceleb_balance\\data npy format\\7465_final_spectro features.npy')

In [ ]:
#for fakeavceleb dataset
bert_features=np.load('F:\\fakeavceleb_balance\\data npy format\\ber\\NEW_features_2.npy')

In [ ]:
All_features1=np.concatenate((bert_features, spec_features,rnn_features), axis=1)

In [ ]:
All_features1.shape

In [ ]:
label=np.load("F:\\fakeavceleb_balance\\data npy format\\ber\\NEW_labels_2.npy")

In [ ]:
label1=[]
for i in range(len(label)):
  if label[i]==0:
    label1.append([1,0])
  else:
    label1.append([0,1])

In [ ]:
label1=np.array(label1)

In [ ]:
model = tensorflow.keras.Sequential()
model.add(Dense(18498, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(Dense(512, activation="relu"))
model.add(layers.Dropout(0.1))
model.add(Dense(128, activation="relu"))
# model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
# model.add(Dense(16, activation="relu"))   
model.add(Dense(8, activation="relu"))
model.add(Dense(units=2, activation="softmax"))

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(All_features1,label1,test_size=0.20,random_state=42)

In [ ]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["acc"])

In [ ]:
history=model.fit(X_train,Y_train,batch_size=64,epochs=50,verbose=1,validation_data=(X_test,Y_test))

In [ ]:
results = model.evaluate(X_test, Y_test)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true and y_pred_binary are your true labels and predicted labels
# y_true should be in multilabel-indicator format, and y_pred_binary should be the predicted binary labels

# Choose an appropriate averaging strategy
# Example: micro, macro, weighted, or None
average_strategy = 'macro'

# Calculate precision, recall, and f1 score with the chosen averaging strategy
precision = precision_score(Y_test, y_pred_binary,average=average_strategy)
recall = recall_score(Y_test,y_pred_binary, average=average_strategy)
f1 = f1_score(Y_test, y_pred_binary, average=average_strategy)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
y_test_1d=Y_test.ravel()
y_pred_binary_1d=y_pred_binary.ravel()

In [ ]:
# Create confusion matrix
cm =confusion_matrix(y_test_1d, y_pred_binary_1d)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

# Plot confusion matrix using seaborn and matplotlib
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=np.unique(y_test_1d), yticklabels=np.unique(y_test_1d))
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

# Compute ROC curve and ROC area for each label
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(Y_test.shape[1]):
    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], y_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
import matplotlib.pyplot as plt

# Plot ROC curve for each label
plt.figure(figsize=(8, 8))
for i in range(Y_test.shape[1]):
    plt.plot(fpr[i], tpr[i], lw=2, label='Label %d (AUC = %0.2f)' % (i, roc_auc[i]))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) for Each Label')
# plt.legend(loc="lower right")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Plot training & validation loss values
# plt.subplot(1, 2, 2)
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')

# plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))

# plt.subplot(1, 2, 1)
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.show()